In [1]:
# %load metamaps.py
"""Provides methods for loading/saving metamaps"""
import struct
from enum import IntEnum
from keras.utils import Sequence
import numpy as np
import os
import random
import numbers

METAMAP_FILE_VERSION = 0x100

class TileType(IntEnum):
    """Tile types in a metamap"""
    UNREACHABLE = 0
    EMPTY = 1
    WALL = 2
    PUSHWALL = 3
    DOOR = 4

class EncodingDim(IntEnum):
    """Dimensions for the one-hot encoding of a metamap"""
    PLAYABLE = 0
    SOLID = 1
    PASSAGE = 2

TileTypeToEncodingDim = {
    TileType.UNREACHABLE: EncodingDim.SOLID,
    TileType.EMPTY: EncodingDim.PLAYABLE,
    TileType.WALL: EncodingDim.SOLID,
    TileType.PUSHWALL: EncodingDim.PASSAGE,
    TileType.DOOR: EncodingDim.PASSAGE,
    }

class MetaMapsSequence(Sequence):
    """A sequence of real metamaps from a directory and randomly generated ones"""

    def __init__(self, maps_dir, batch_size):
        self.maps_dir = maps_dir
        self.batch_size = batch_size
        self.map_files = os.listdir(maps_dir)
        NUM_MAPS = len(self.map_files)
        real_maps = [(index, True) for index in range(NUM_MAPS)]
        fake_maps = [(index + NUM_MAPS, False) for index in range(NUM_MAPS)]
        map_order = real_maps + fake_maps
        np.random.shuffle(map_order)
        self.map_order = map_order

    def __len__(self):
        return int(len(self.map_order) / self.batch_size)

    def __getitem__(self, idx):
        map_batch = np.zeros((self.batch_size, 64, 64, len(EncodingDim)))
        label_batch = np.zeros((self.batch_size))

        for i in range(self.batch_size):
            (index, real_map) = self.map_order[idx * self.batch_size + i]
            if real_map:
                label_batch[i] = 1
                map_batch[i:] = load_metamap(
                    os.path.join(self.maps_dir, self.map_files[index]))
            else:
                map_batch[i:] = generate_random_map()

        return map_batch, label_batch


def generate_random_map():
    """Generate a random map"""
    width = 64
    height = 64
    size = width * height

    junk_map = np.zeros([size, len(EncodingDim)])

    for i in range(size):
        tile_type = random.randint(0, len(EncodingDim) - 1)
        junk_map[i, tile_type] = 1

    junk_map.shape = (width, height, len(EncodingDim))

    return junk_map


def load_smashed_metamap(filename):
    """Loads a condensed metamap file"""
    with open(filename, "rb") as fin:
        mega_meta_map = np.fromfile(fin, dtype=np.uint8)
    return mega_meta_map


def load_all_metamaps(dirname, number_cap=None):
    """Loads all the metamaps in the given directory, returning a giant numpy array"""
    map_names = os.listdir(dirname)
    if isinstance(number_cap, numbers.Integral) and number_cap > 0:
        map_names = map_names[:number_cap]
    all_maps = np.zeros((len(map_names), 64, 64, len(EncodingDim)))

    for index, map_name in enumerate(map_names):
        load_metamap_into(os.path.join(dirname, map_name), all_maps, index)

    return all_maps


def load_metamap_into(filename, all_maps, index):
    """Loads a metamap from a file into a numpy array of shape (width, height, 3)"""
    with open(filename, "rb") as fin:
        version = struct.unpack('Q', fin.read(8))[0]

        if version != METAMAP_FILE_VERSION:
            raise ValueError("Unsupported metamap version")

        width = struct.unpack('i', fin.read(4))[0]
        height = struct.unpack('i', fin.read(4))[0]

        raw_map = np.fromfile(fin, dtype=np.uint8)
        raw_map.shape = (width, height)

        for y in range(height):
            for x in range(width):
                tile_type = TileType(raw_map[y, x])
                all_maps[index, y, x, TileTypeToEncodingDim[tile_type]] = 1


def load_metamap(filename):
    """Loads a metamap from a file into a numpy array of shape (width, height, 3)"""
    with open(filename, "rb") as fin:
        version = struct.unpack('Q', fin.read(8))[0]

        if version != METAMAP_FILE_VERSION:
            raise ValueError("Unsupported metamap version")

        width = struct.unpack('i', fin.read(4))[0]
        height = struct.unpack('i', fin.read(4))[0]
        size = width * height

        raw_map = np.fromfile(fin, dtype=np.uint8)
        one_hot = np.zeros([size, len(EncodingDim)])

        for i in range(size):
            tile_type = TileType(raw_map[i])
            one_hot[i, TileTypeToEncodingDim[tile_type]] = 1

        one_hot.shape = (width, height, len(EncodingDim))

        return one_hot


def save_metamap(metamap, filename):
    """Saves a metamap to a file"""
    with open(filename, "wb") as fout:
        fout.write(struct.pack('Q', METAMAP_FILE_VERSION))

        width = metamap.shape[0]
        height = metamap.shape[1]

        fout.write(struct.pack('i', width))
        fout.write(struct.pack('i', height))
        for y in range(height):
            for x in range(width):
                tile_type = TileType.WALL
                if metamap[y, x, EncodingDim.PLAYABLE] == 1:
                    tile_type = TileType.EMPTY
                elif metamap[y, x, EncodingDim.SOLID] == 1:
                    tile_type = TileType.WALL
                elif metamap[y, x, EncodingDim.PASSAGE] == 1:
                    tile_type = TileType.DOOR

                fout.write(struct.pack('b', tile_type))
    return

C:\Users\david\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

In [3]:
mega = load_smashed_metamap("MegaMetaMap").astype("bool")
print(f"Mega shape: {mega.shape}")

Mega shape: (420950016,)


In [4]:
WIDTH = 64
HEIGHT = 64
ALPHABET = len(EncodingDim)

In [5]:
mega.shape = (int(mega.shape[0]/ALPHABET), ALPHABET)
print(f"Mega reshaped: {mega.shape}")

Mega reshaped: (140316672, 3)


In [14]:
num_rows = int(mega.shape[0]/WIDTH)
num_maps = int(mega.shape[0]/WIDTH/HEIGHT)
print(f"Number of rows: {num_rows:,d}")
print(f"Number of maps: {num_maps:,d}")

Number of rows: 2,192,448
Number of maps: 34,257


In [7]:
sampling_step = 3
samples_in_map = int((WIDTH * (HEIGHT - 1) - 1) / sampling_step)

print(f"Samples in map: {samples_in_map:,d}")

Samples in map: 1,343


In [8]:
total_samples = samples_in_map * num_maps
print(f"Total samples: {total_samples:,d}")

Total samples: 46,007,151


# Vectorizing

In [9]:
sentences = np.zeros((total_samples, WIDTH, ALPHABET), dtype=np.bool)
next_tiles = np.zeros((total_samples, ALPHABET), dtype=np.bool)

In [11]:
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [13]:
sentences_byte_size = sentences.size * sentences.dtype.itemsize
next_tiles_byte_size = next_tiles.size * next_tiles.dtype.itemsize
print(f"Sentences size: {sizeof_fmt(sentences_byte_size)}")
print(f"Next Tiles size: {sizeof_fmt(next_tiles_byte_size)}")
print(f"Total: {sizeof_fmt(sentences_byte_size + next_tiles_byte_size)}")

Sentences size: 8.2GiB
Next Tiles size: 131.6MiB
Total: 8.4GiB


In [ ]:
for map_num in range(num_maps):
    map_offset = map_num * WIDTH * HEIGHT 
    for sample_index in range(samples_in_map):
        start_offset = map_offset + sample_index * sampling_step
        sentence_index = map_num*samples_in_map + sample_index
        sentences[sentence_index] = mega[start_offset: start_offset + WIDTH]
        next_tiles[sentence_index] = mega[start_offset + WIDTH]

# Create Model

In [ ]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(WIDTH, ALPHABET)))
model.add(layers.Dense(ALPHABET, activation='softmax'))

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Training and Generation

In [ ]:
# TODO: This still needs work
# Need some way to display a map (text? image?)
# Force it sample stuff into a generated map of some sort

import datetime
import random
import sys

print(f"Started: {datetime.datetime.time(datetime.datetime.now())}")

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(sentences, next_tiles, batch_size=128, epochs=1)
    start_index = random.randint(0, num_rows) * WIDTH
    generated_map = text[start_index: start_index + WIDTH]
    print('--- Generating with seed: "' + generated_map + '"')
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        print()
        sys.stdout.write(generated_map)

        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
        print()

print(f"Done: {datetime.datetime.time(datetime.datetime.now())}")